<a href="https://colab.research.google.com/github/osmarbraz/sri/blob/main/4_NER_spaCy_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconhecimento de entidade nomeada utilizando Spacy com Modelo em Português

Exemplo do uso Spacy Python Library com execução através do Google Colaboratory para reconhecimento de entidade nomeada.<br>

Funciona no Jupiter Notebook com algumas modificações.

Site ferramenta: https://spacy.io/

Tutorial<br>
https://medium.com/@van3ssabandeira/o-famoso-spacy-90afb683b6fe


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [1]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório do notebook no diretório corrente.   


In [2]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioNotebook():
    """
      Verifica se existe o diretório do notebook no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_NOTEBOOK):  
        # Cria o diretório
        os.makedirs(DIRETORIO_NOTEBOOK)
        logging.info("Diretório do notebook criado: {}".format(DIRETORIO_NOTEBOOK))
    
    return DIRETORIO_NOTEBOOK

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [3]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [4]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModelosParametros:
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )

Biblioteca de limpeza de tela


In [5]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [6]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [7]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [8]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [9]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.
    
    Retorno:
    `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [10]:
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Conecta o modelo ao device

In [11]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.       
        `device` - Um device de GPU.     
    
      Retorno:
        `model` - Um objeto model BERT conectado a GPU.     
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():    
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()
        
    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [12]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [13]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")     

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.7 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [14]:
# Instala dependências do spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# Instala uma versão específica
!pip install -U spacy==3.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 Parametrização

## Gerais

In [16]:
# Definição dos parâmetros a serem avaliados

## Específicos

Parâmetros do modelo

In [17]:
# Definição dos parâmetros do Modelo.
model_args = ModelosParametros(     

    #modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",
    #modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    modelo_spacy = "pt_core_news_sm",
        
)

## Nome do diretório dos arquivos de dados

In [18]:
# Diretório do notebook
DIRETORIO_NOTEBOOK = "SRI"

## Define o caminho para os arquivos de dados

In [19]:
# Diretório local para os arquivos de dados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_NOTEBOOK + "/"

# Diretório no google drive com os arquivos de dados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/" + DIRETORIO_NOTEBOOK + "/data/"

# 3 spaCy

## 3.1 Download arquivo modelo

Uso:
https://spacy.io/usage

Modelos:
https://spacy.io/models

In [20]:
!python -m spacy download $model_args.modelo_spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-02 17:45:17.623540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-02 17:45:17.623610: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-02 17:45:17.623622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

## 3.2 Carrega o modelo

In [21]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

nlp = spacy.load(model_args.modelo_spacy)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# 4 Exemplos de uso

## 4.1 Carregamento do Dataset

### 4.1.1 Especifica os nomes dos arquivos de dados



In [22]:
# Nome do arquivo
NOME_ARQUIVO_DATASET = "dataset.csv"
NOME_ARQUIVO_DATASET_COMPACTADO = "dataset.zip"
NOME_ARQUIVO_DATASET_POS = "datasetpos.csv"
NOME_ARQUIVO_DATASET_POS_COMPACTADO = "datasetpos.zip"

### 4.1.2 Cria o diretório local para receber os dados

In [23]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

INFO:root:Diretório já existe: /content/SRI


### 4.1.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [24]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_DATASET_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_DATASET_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


Descompacta os arquivos.

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [25]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

INFO:root:Terminei a descompactação.


### 4.1.4 Carregamento das lista com os dados dos arquivos e postagging

#### Carrega o arquivo dos dados e POS

In [26]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_dataset = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET, sep=";", encoding="UTF-8")
df_dataset_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO DOCUMENTOS: {}.".format(len(df_dataset)))
logging.info("TERMINADO DOCUMENTOS POS: {}.".format(len(df_dataset_pos)))

INFO:root:TERMINADO DOCUMENTOS: 20.
INFO:root:TERMINADO DOCUMENTOS POS: 20.


In [27]:
df_dataset.sample(5)

,id,sentencas,documento
4,5,"['A seleção brasileira masculina de vôlei, que...","A seleção brasileira masculina de vôlei, que é..."
10,11,"['Nove pessoas morreram, sendo três crianças, ...","Nove pessoas morreram, sendo três crianças, e ..."
15,16,['O Brasil lavou a alma após o decepcionante e...,O Brasil lavou a alma após o decepcionante emp...
6,7,['A aviação israelense atacou 150 alvos na mad...,A aviação israelense atacou 150 alvos na madru...
2,3,"['O ministro da Defesa, Nelson Jobim, informou...","O ministro da Defesa, Nelson Jobim, informou n..."


In [28]:
df_dataset_pos.sample(5)

,id,pos_documento
18,19,"[[['A', 'ginasta', 'Jade', 'Barbosa', ',', 'qu..."
6,7,"[[['A', 'aviação', 'israelense', 'atacou', '15..."
1,2,"[[['Depois', 'de', '20', 'dias', 'de', 'tempo'..."
4,5,"[[['A', 'seleção', 'brasileira', 'masculina', ..."
3,4,"[[['Usando', 'telescópios', 'do', 'Observatóri..."


#### Corrigir os tipos de colunas dos dados e POS

Em dados:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [29]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
df_dataset["sentencas"] = df_dataset["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

df_dataset_pos["pos_documento"] = df_dataset_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO DOCUMENTOS: {}.".format(len(df_dataset)))
logging.info("TERMINADO CORREÇÃO DOCUMENTOS POS: {}.".format(len(df_dataset_pos)))

INFO:root:TERMINADO CORREÇÃO DOCUMENTOS: 20.
INFO:root:TERMINADO CORREÇÃO DOCUMENTOS POS: 20.


#### Criando dados indexados 

In [30]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
df_dataset_indexado = df_dataset.set_index(["id"])
df_dataset_indexado.head()

,sentencas,documento
id,,
1,[Ao menos 17 pessoas morreram após a queda de ...,Ao menos 17 pessoas morreram após a queda de u...
2,"[Depois de 20 dias de tempo seco, voltou a cho...","Depois de 20 dias de tempo seco, voltou a chov..."
3,"[O ministro da Defesa, Nelson Jobim, informou ...","O ministro da Defesa, Nelson Jobim, informou n..."
4,[Usando telescópios do Observatório Europeu Su...,Usando telescópios do Observatório Europeu Sul...
5,"[A seleção brasileira masculina de vôlei, que ...","A seleção brasileira masculina de vôlei, que é..."


In [31]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
df_dataset_pos_indexado = df_dataset_pos.set_index(["id"])
df_dataset_pos_indexado.head()

,pos_documento
id,
1,"[[[Ao, menos, 17, pessoas, morreram, após, a, ..."
2,"[[[Depois, de, 20, dias, de, tempo, seco, ,, v..."
3,"[[[O, ministro, da, Defesa, ,, Nelson, Jobim, ..."
4,"[[[Usando, telescópios, do, Observatório, Euro..."
5,"[[[A, seleção, brasileira, masculina, de, vôle..."


## 4.2 Exemplo 1

## Texto a ser analisado

In [32]:
# Recupera o primeiro documento
documento = df_dataset['documento'][0]

# Ou especifique diretamente um texto
#documento = "O Brasil, um vasto país sul-americano, estende-se da Bacia Amazônica, no norte, até os vinhedos e as gigantescas Cataratas do Iguaçu, no sul. O Rio de Janeiro, simbolizado pela sua estátua de 38 metros de altura do Cristo Redentor, situada no topo do Corcovado, é famoso pelas movimentadas praias de Copacabana e Ipanema, bem como pelo imenso e animado Carnaval, com desfiles de carros alegóricos, fantasias extravagantes e samba."

# Mostra o documento
print(documento)

Ao menos 17 pessoas morreram após a queda de um avião de passageiros na República Democrática do Congo. Todos morreram quando o avião, prejudicado pelo mau tempo, não conseguiu chegar à pista de aterrissagem e caiu numa floresta a 15 quilômetros do aeroporto de Bukavu. Ele havia saído da cidade mineira de Lugushwa em direção a Bukavu, numa distância de 130 quilômetros.


### Armazena um texto.

In [33]:
# Submete o texto ao spaCy
doc = nlp(documento)

logging.info('Texto armazenado!')

INFO:root:Texto armazenado!


### Identificando as entidades. (NER/Named Entity Recognition)

**Text**: o texto original da entidade.<br>
**Start**: Índice de início da entidade no Doc.<br>
**End**: Índice do fim da entidade no Doc.<br>
**Label**: rótulo da entidade, ou seja, tipo.<br>

https://medium.com/botsbrasil/como-reconhecer-entidades-na-l%C3%ADngua-portuguesa-usando-spacy-8a5ca6f42c4f

### Listando as entidades identificadas

In [34]:
for ent in doc.ents:
   print(ent.text, ent.start_char, ent.end_char, ent.label_)

República Democrática do Congo 72 102 LOC
aeroporto de Bukavu 249 268 LOC
Lugushwa 307 315 LOC


ou

In [35]:
[(entity, entity.label_) for entity in doc.ents]

[(República Democrática do Congo, 'LOC'),
 (aeroporto de Bukavu, 'LOC'),
 (Lugushwa, 'LOC')]

### Visualizando o texto marcado

In [36]:
# Importando as bibliotecas.
import spacy 
from spacy  import displacy

displacy.render(doc,style="ent",jupyter=True)

## 4.3 Exemplo 2

## Texto a ser analisado

In [37]:
# Recupera o primeiro documento
documento = df_dataset['documento'][1]

# Ou especifique diretamente um texto
#documento = "O Brasil, um vasto país sul-americano, estende-se da Bacia Amazônica, no norte, até os vinhedos e as gigantescas Cataratas do Iguaçu, no sul. O Rio de Janeiro, simbolizado pela sua estátua de 38 metros de altura do Cristo Redentor, situada no topo do Corcovado, é famoso pelas movimentadas praias de Copacabana e Ipanema, bem como pelo imenso e animado Carnaval, com desfiles de carros alegóricos, fantasias extravagantes e samba."

# Mostra o documento
print(documento)

Depois de 20 dias de tempo seco, voltou a chover na capital paulista. A forte chuva em São Paulo complicava o trânsito na manhã desta segunda-feira, 16, e fez com que o Centro de Gerenciamento de Emergência (CGE) da Prefeitura colocasse a cidade em estado de atenção. Até 9h30m foram registrados oito pontos de alagamento, dois deles intransitáveis - na Marginal Pinheiros, na altura da Ponte João Dias, e na Marginal Tietê, no acesso à Rodovia dos Bandeirantes. Às 9 horas, a cidade tinha 113 km de lentidão, sendo que a média para o horário é de 82 km, segundo a Companhia de Engenharia de Tráfego (CET).


### Armazena um texto.

In [38]:
# Submete o texto ao spaCy
doc = nlp(documento)

logging.info('Texto armazenado!')

INFO:root:Texto armazenado!


### Identificando as entidades. (NER/Named Entity Recognition)

**Text**: o texto original da entidade.<br>
**Start**: Índice de início da entidade no Doc.<br>
**End**: Índice do fim da entidade no Doc.<br>
**Label**: rótulo da entidade, ou seja, tipo.<br>

https://medium.com/botsbrasil/como-reconhecer-entidades-na-l%C3%ADngua-portuguesa-usando-spacy-8a5ca6f42c4f

### Listando as entidades identificadas

In [39]:
for ent in doc.ents:
   print(ent.text, ent.start_char, ent.end_char, ent.label_)

São Paulo 87 96 LOC
Centro de Gerenciamento de Emergência 169 206 LOC
Prefeitura 216 226 LOC
Marginal 354 362 LOC
Ponte João Dias 387 402 LOC
Marginal Tietê 409 423 LOC
Rodovia dos Bandeirantes 437 461 LOC
Companhia de Engenharia de Tráfego 565 599 ORG


ou

In [40]:
[(entity, entity.label_) for entity in doc.ents]

[(São Paulo, 'LOC'),
 (Centro de Gerenciamento de Emergência, 'LOC'),
 (Prefeitura, 'LOC'),
 (Marginal, 'LOC'),
 (Ponte João Dias, 'LOC'),
 (Marginal Tietê, 'LOC'),
 (Rodovia dos Bandeirantes, 'LOC'),
 (Companhia de Engenharia de Tráfego, 'ORG')]

### Visualizando o texto marcado

In [41]:
# Importando as bibliotecas.
import spacy 
from spacy  import displacy

displacy.render(doc,style="ent",jupyter=True)

# 5 Finalização

## 5.1 Tempo final de processamento



In [42]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:00:40 (h:mm:ss)
